In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'http://haoetf.com/'

html = requests.get(url).content.decode('utf-8')

soup = BeautifulSoup(html,features='lxml')

soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>\n  <meta content="HaoETF,理财,量化投资,可转债,分级基金,基金,ETF,LOF,QDII" name="keywords"/>\n  <meta content="HaoETF，理财源自数据" name="description"/>\n  <title>\n   HaoETF 理财源自数据\n  </title>\n  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@4.4.1/dist/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" rel="stylesheet"/>\n  <!-- Favicons -->\n  <link href="/static/img/favicons/favicon-180x180.png" rel="apple-touch-icon" sizes="180x180"/>\n  <link href="/static/img/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>\n  <link href="/static/img/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>\n  <link href="/static/img/favicons/favicon.ico" rel="icon"/>\n  <style>\n   .bd-placeholder-img {\nfont-size: 1.125rem;\ntext-anchor:

In [5]:
#获取表头
thead = soup.body.table.thead.tr

thead_list = []
for th in thead.contents :
    if th.string != '\n' :
        thead_list.append(th.string)

thead_list.remove('<th>估值误差</th>')

thead_list

['代码',
 '名称',
 '估值溢价',
 '实时溢价',
 '现价',
 '涨跌',
 '成交量(万手)',
 '成交额(万元)',
 '净值',
 '净值日期',
 'T-1估值',
 '估值日期',
 '估值涨跌',
 '仓位估算',
 '限购(元)']

In [6]:
tbody = soup.body.table.tbody

tr_list = []
for tr in tbody.find_all('tr'):
    td_list = []
    for td in tr.find_all('td'):
        td_list.append(td.string)
    tr_list.append(td_list)

tr_list


[['160216',
  '国泰商品',
  '15.70%',
  '6.92%',
  '0.250',
  '-3.10%',
  '85.64',
  '2146.34',
  '0.249',
  '2020-03-17',
  '0.216',
  '2020-03-18',
  '-13.51%',
  '77.13%',
  '暂停申购'],
 ['160416',
  '石油基金',
  '0.88%',
  '-4.60%',
  '0.566',
  '-1.57%',
  '165.69',
  '9342.23',
  '0.601',
  '2020-03-17',
  '0.561',
  '2020-03-18',
  '-6.95%',
  '53.15%',
  '500000'],
 ['160723',
  '嘉实原油',
  '9.83%',
  '5.43%',
  '0.723',
  '0.84%',
  '203.24',
  '14618.50',
  '0.703',
  '2020-03-17',
  '0.658',
  '2020-03-18',
  '-6.63%',
  '37.86%',
  '暂停申购'],
 ['161129',
  '原油基金',
  '38.26%',
  '30.71%',
  '0.767',
  '1.19%',
  '17.78',
  '1354.23',
  '0.633',
  '2020-03-17',
  '0.555',
  '2020-03-18',
  '-12.71%',
  '53.43%',
  '暂停申购'],
 ['162411',
  '华宝油气',
  '32.76%',
  '22.92%',
  '0.223',
  '0.45%',
  '343.87',
  '7675.60',
  '0.180',
  '2020-03-17',
  '0.168',
  '2020-03-18',
  '-7.20%',
  '75.11%',
  '暂停申购'],
 ['162719',
  '广发石油',
  '-0.31%',
  '-6.68%',
  '0.445',
  '-1.77%',
  '56.14',
  '2489.6

In [8]:
#创建表
table = pd.DataFrame(tr_list,columns=thead_list)
#设定溢价率变量
discount_rt = thead_list[2]

#去除百分比%字符
table[discount_rt] = table[discount_rt].str.replace('%','')
#将字符串转化为数字格式
table[discount_rt] = pd.to_numeric(table[discount_rt],errors='ignore')
table['成交额(万元)'] = pd.to_numeric(table['成交额(万元)'],errors='ignore')

#选取溢价率超过2%且成交额>500万且不开放申购的基金
table = table[(table[discount_rt] > 2)  & 
              (table['限购(元)'].str.contains('暂停') == False) &
              (table['成交额(万元)'] > 500)].sort_values(discount_rt,ascending=False)

table

,代码,名称,估值溢价,实时溢价,现价,涨跌,成交量(万手),成交额(万元),净值,净值日期,T-1估值,估值日期,估值涨跌,仓位估算,限购(元)


In [10]:
table = table.loc[:,['代码','名称',discount_rt,'现价','T-1估值']]

#编辑haoETF原油基金估值详情
oil_desp = '基金代码 | 基金名称 | 溢价率 | 场内现价 | 场外估值 ' +'\n\n'
for index,oil in table.iterrows():
    oil_desp = oil_desp + oil['代码'] + ' | ' + oil['名称'] + ' | ' + str(oil[discount_rt])+'% | ' + oil['现价'] +' | '+oil['T-1估值'] + '\n\n'

oil_desp

'基金代码 | 基金名称 | 溢价率 | 场内现价 | 场外估值 \n\n'